# Fine-tune Qwen3-1.7B with LoRA

This notebook guide provides a comprehensive overview of using the `transformers` Python package to efficiently train a custom model. It covers the following techniques:

1. Utilizing model, tokenizer, and dataset loading functionalities from Hugging Face.
2. Loading and preprocessing custom dataset.
3. Training the model with QLoRA (4-bit quantization + LoRA).
4. Evaluating the model's performance on validation set.
5. Saving your custom model and preparing it for deployment.

## Preliminary Preparation

Before proceeding with model training, ensure your environment is properly configured by following these steps:

1. Install the necessary Python packages.
2. Import the required libraries.

In [ ]:

# 安装核心依赖
!pip install -q protobuf==3.20.3
!pip install -q h5py typing-extensions wheel
!pip install -q -U bitsandbytes
!pip install -q -U transformers==4.53
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets
!pip install -q matplotlib
!pip install -q pandas
!pip install -q fschat

print("=" * 70)
print("✅ 依赖安装完成！")
print("⚠️  如果遇到protobuf警告，请重启kernel后继续运行")
print("=" * 70)

In [ ]:
!nvidia-smi

In [ ]:
# Set Hugging Face cache directory to Kaggle working directory
import os

cache_dir = '/kaggle/working/huggingface_cache'
os.makedirs(cache_dir, exist_ok=True)
os.environ['HF_HOME'] = cache_dir
os.environ['TRANSFORMERS_CACHE'] = f'{cache_dir}/transformers'
os.environ['HF_DATASETS_CACHE'] = f'{cache_dir}/datasets'

print("=" * 70)
print(f"Hugging Face 缓存目录已设置为: {cache_dir}")
print(f"模型和数据集将保存到Kaggle工作目录")
print("=" * 70)

## Load Pre-trained model and tokenizer

First let's load the model we are going to use - Qwen3-1.7B! This is a smaller model suitable for efficient fine-tuning. The model is available at https://huggingface.co/Qwen/Qwen3-1.7B

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import os

# Quantization type (fp4 or nf4), According to QLoRA paper, for training 4-bit base models (e.g. using LoRA adapters) one should use
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = True

model_id = "Qwen/Qwen3-1.7B"

# Check if model is already cached in Kaggle
# Hugging Face stores models in snapshots directory
model_cache_path = os.path.join(
    os.environ.get('TRANSFORMERS_CACHE', ''),
    'models--Qwen--Qwen3-1_7B'
)

def check_model_cached(cache_path):
    """Check if model files actually exist in cache"""
    if not os.path.exists(cache_path):
        return False
    
    # Check for snapshots directory
    snapshots_path = os.path.join(cache_path, 'snapshots')
    if not os.path.exists(snapshots_path):
        return False
    
    # Check if any snapshot directory has ALL model weight files
    try:
        snapshot_dirs = os.listdir(snapshots_path)
        if not snapshot_dirs:
            return False
        
        # Check the most recent snapshot for model files
        for snapshot_dir in snapshot_dirs:
            snapshot_full_path = os.path.join(snapshots_path, snapshot_dir)
            if os.path.isdir(snapshot_full_path):
                files = os.listdir(snapshot_full_path)
                # Look for safetensors or bin files
                model_files = [f for f in files if f.endswith('.safetensors') or f.endswith('.bin')]
                
                # Qwen3-1.7B should have model files
                # Check if we have at least one complete model file > 100MB to verify it's actually downloaded
                for model_file in model_files:
                    file_path = os.path.join(snapshot_full_path, model_file)
                    if os.path.exists(file_path):
                        file_size = os.path.getsize(file_path)
                        # If we find a large model file (> 100MB), cache is likely complete
                        if file_size > 100 * 1024 * 1024:  # 100MB
                            return True
        return False
    except Exception as e:
        print(f"⚠️  检查缓存时出错: {e}")
        return False

print("=" * 70)
if check_model_cached(model_cache_path):
    print("✅ 检测到已缓存完整模型")
    print(f"📂 缓存位置: {model_cache_path}")
    print("⚡ 将从缓存加载，无需重新下载")
else:
    print("⚠️  未检测到完整模型缓存")
    print("📥 首次下载模型 (约 3-4 GB)，请耐心等待...")
    print(f"💾 模型将保存到: {model_cache_path}")
print("=" * 70)

# Kaggle GPU支持4-bit量化
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=use_nested_quant,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=torch.bfloat16
)

print("\n" + "=" * 70)
print("🔄 加载 Tokenizer...")

# Get the cache directory from environment variable
transformers_cache_dir = os.environ.get('TRANSFORMERS_CACHE', '')

# Check if tokenizer is cached
def check_tokenizer_cached(cache_path):
    """Check if tokenizer files exist in cache"""
    if not os.path.exists(cache_path):
        return False
    
    snapshots_path = os.path.join(cache_path, 'snapshots')
    if not os.path.exists(snapshots_path):
        return False
    
    try:
        snapshot_dirs = os.listdir(snapshots_path)
        if not snapshot_dirs:
            return False
        
        for snapshot_dir in snapshot_dirs:
            snapshot_full_path = os.path.join(snapshots_path, snapshot_dir)
            if os.path.isdir(snapshot_full_path):
                files = os.listdir(snapshot_full_path)
                # Look for tokenizer files
                has_tokenizer = any(f.startswith('tokenizer') for f in files)
                if has_tokenizer:
                    return True
        return False
    except:
        return False

if check_tokenizer_cached(model_cache_path):
    print("✅ Tokenizer 已缓存，从本地加载")
else:
    print("📥 首次下载 Tokenizer 文件...")

# 显式指定 cache_dir
tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir=transformers_cache_dir)

# Fix: Set pad_token to avoid attention_mask warnings
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id
    print(f"设置 pad_token = eos_token ({tokenizer.pad_token})")
print("=" * 70)

print("\n" + "=" * 70)
print("🔄 加载模型...")
# Kaggle GPU支持量化和device_map
model = AutoModelForCausalLM.from_pretrained(
    model_id, 
    quantization_config=bnb_config,
    device_map={"":0},
    cache_dir=transformers_cache_dir
)
print("=" * 70)

# Resize token embeddings to match tokenizer
model.resize_token_embeddings(len(tokenizer))
print(f"\n✅ 模型加载完成，vocabulary size: {len(tokenizer)}")

## Data Loading and Preprocessing

This section covers the complete data pipeline:
1. Loading the custom dataset from local files
2. Converting data to conversation format
3. Creating Hugging Face Dataset
4. Splitting into train/validation sets
5. Implementing custom dataset class for training


In [ ]:
import json
import os
import glob

# 在Kaggle中查找数据集文件
# 支持多种可能的数据集路径
possible_paths = [
    "/kaggle/input/nlp-final/generated_dataset.json",      # 最可能的路径
    "/kaggle/input/nlp-final/data/generated_dataset.json",  # 如果文件在data子目录
    "/kaggle/input/nlp-final/*/generated_dataset.json",     # 通配符搜索
    "/kaggle/input/*/generated_dataset.json",               # 更广泛的搜索
    "data/generated_dataset.json",                          # 本地测试路径
    "generated_dataset.json"                                # 当前目录
]

dataset_file = None
for path in possible_paths:
    if '*' in path:
        # 使用glob查找
        matches = glob.glob(path)
        if matches:
            dataset_file = matches[0]
            break
    elif os.path.exists(path):
        dataset_file = path
        break

print("=" * 80)
if dataset_file:
    print(f"✅ 找到数据集文件: {dataset_file}")
    print("=" * 80)
    with open(dataset_file, 'r', encoding='utf-8') as f:
        raw_data = json.load(f)
    print(f"✅ 数据集加载完成: {len(raw_data)} 条")
    print(f"\n📝 示例数据:")
    print(f"   问题: {raw_data[0]['question'][:50]}...")
    print(f"   答案: {str(raw_data[0]['answer'])[:80]}...")
else:
    print("⚠️  未找到数据集文件 generated_dataset.json")
    print("=" * 80)
    print("\n💡 搜索的路径:")
    for path in possible_paths:
        print(f"   • {path}")
    print("\n❌ 请确保在Kaggle Notebook中添加 'nlp-final' dataset")
    raise FileNotFoundError("无法找到 generated_dataset.json 文件")
print("=" * 80)


In [ ]:
# Convert the dataset to the required format
# The dataset has 'question' and 'answer' fields, where answer is a JSON array
# We need to convert answer to string format for training
converted_data = []
for item in raw_data:
    # Convert answer (JSON array) to string format
    answer_str = json.dumps(item['answer'], ensure_ascii=False)
    converted_data.append({
        "conversations": [
            {"role": "human", "value": item['question']},
            {"role": "gpt", "value": answer_str}
        ]
    })

print(f"✅ 数据转换完成: {len(converted_data)} 条")
print(f"\n📝 转换后示例:")
print(f"   问题: {converted_data[0]['conversations'][0]['value'][:50]}...")
print(f"   答案: {converted_data[0]['conversations'][1]['value'][:80]}...")

In [ ]:
# Display data sample information
print(f"数据样本预览（共 {len(converted_data)} 条）：")
print(f"   问题: {converted_data[0]['conversations'][0]['value'][:50]}...")
print(f"   答案: {converted_data[0]['conversations'][1]['value'][:80]}...")

In [ ]:
from datasets import Dataset, load_from_disk
import os

# 定义数据集的保存路径
dataset_path = "/kaggle/working/huggingface_cache/custom_dataset"

# 检查是否已有保存的数据集
if os.path.exists(dataset_path):
    print("=" * 80)
    print("✅ 发现已保存的数据集，直接加载...")
    print(f"📂 位置: {dataset_path}")
    dataset = load_from_disk(dataset_path)
    print(f"✅ 数据集加载完成: {len(dataset)} 条")
    print("=" * 80)
else:
    print("=" * 80)
    print("📦 创建 Hugging Face Dataset...")
    print("=" * 80)
    
    # Create Hugging Face Dataset from converted data
    dataset = Dataset.from_list(converted_data)
    print(f"✅ 数据集创建完成: {len(dataset)} 条")
    
    # 💾 保存数据集到磁盘
    print(f"\n💾 保存数据集到: {dataset_path}")
    os.makedirs(os.path.dirname(dataset_path), exist_ok=True)
    dataset.save_to_disk(dataset_path)
    print("✅ 数据集已保存！下次运行将直接加载")
    print("=" * 80)


In [ ]:
from torch.utils.data import random_split

# 按比例划分数据集（更灵活，自动适应数据集大小）
train_ratio = 0.8   # 80% 用于训练

total_size = len(dataset)
train_size = int(total_size * train_ratio)
val_size = total_size - train_size  # 确保总和等于 total_size，避免舍入误差

print(f"📊 数据集划分:")
print(f"   总数据量: {total_size} 条")
print(f"   训练集: {train_size} 条 ({train_size/total_size*100:.1f}%)")
print(f"   验证集: {val_size} 条 ({val_size/total_size*100:.1f}%)")

# 只分成两部分：训练集和验证集
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])


### Customized Dataset Class

Create a specialized dataset class named "InstructDataset" designed to handle our custom dataset format with proper tokenization and label masking.


In [ ]:
import json, copy
import transformers
from typing import Dict, Sequence, List
from dataclasses import dataclass
from torch.utils.data import Dataset
from fastchat.conversation import get_conv_template

IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = "<pad>"
DEFAULT_BOS_TOKEN = "<s>"
DEFAULT_EOS_TOKEN = "</s>"
DEFAULT_UNK_TOKEN = "<unk>"
default_conversation = get_conv_template('phoenix')

class InstructDataset(Dataset):
    def __init__(self, data: Sequence, tokenizer: transformers.PreTrainedTokenizer) -> None:
        super().__init__()
        self.tokenizer = tokenizer
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index) -> Dict[str, torch.Tensor]:
        sources = self.data[index]
        if isinstance(index, int):
            sources = [sources]
        data_dict = preprocess([e['conversations'] for e in sources], self.tokenizer)
        if isinstance(index, int):
            data_dict = dict(input_ids=data_dict["input_ids"][0], labels=data_dict["labels"][0])
        return data_dict

def preprocess(
        sources: Sequence[str],
        tokenizer: transformers.PreTrainedTokenizer,
        max_length=1024
) -> Dict:
    # add end signal and concatenate together
    conversations = []
    intermediates = []
    for source in sources:
        header = f"{default_conversation.system_message}"
        conversation, intermediate = _add_speaker_and_signal(header, source)
        conversations.append(conversation)
        intermediates.append(intermediate)

    # tokenize conversations
    conversations_tokenized = _tokenize_fn(conversations, tokenizer)
    input_ids = conversations_tokenized["input_ids"]
    targets = copy.deepcopy(input_ids)

    # keep only machine responses as targets
    assert len(targets) == len(intermediates)
    for target, inters in zip(targets, intermediates):
        mask = torch.zeros_like(target, dtype=torch.bool)
        for inter in inters:
            tokenized = _tokenize_fn(inter, tokenizer)
            start_idx = tokenized["input_ids"][0].size(0) - 1
            end_idx = tokenized["input_ids"][1].size(0)
            mask[start_idx:end_idx] = True
        target[~mask] = IGNORE_INDEX

    input_ids = input_ids[:max_length]
    targets = targets[:max_length]
    return dict(input_ids=input_ids, labels=targets)

def _add_speaker_and_signal(header, source, get_conversation=True):
    BEGIN_SIGNAL = DEFAULT_BOS_TOKEN
    END_SIGNAL = DEFAULT_EOS_TOKEN
    conversation = header
    intermediate = []
    for sentence in source:
        from_str = sentence["role"]
        if from_str.lower() == "human":
            from_str = default_conversation.roles[0]
        elif from_str.lower() == "gpt":
            from_str = default_conversation.roles[1]
        else:
            from_str = 'unknown'
        # store the string w/o and w/ the response
        value = (from_str + ": " + BEGIN_SIGNAL + sentence["value"] + END_SIGNAL)
        if sentence["role"].lower() == "gpt":
            start = conversation + from_str + ": " + BEGIN_SIGNAL
            end = conversation + value
            intermediate.append([start, end])
        if get_conversation:
            conversation += value
    return conversation, intermediate

def _tokenize_fn(strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
    tokenized_list = [
        tokenizer(
            text,
            return_tensors="pt",
            padding="longest",
            max_length=tokenizer.model_max_length,
            truncation=True,
        ) for text in strings
    ]
    input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
    input_ids_lens = labels_lens = [
        tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item()
        for tokenized in tokenized_list
    ]
    return dict(
        input_ids=input_ids,
        labels=labels,
        input_ids_lens=input_ids_lens,
        labels_lens=labels_lens,
    )

@dataclass
class DataCollatorForSupervisedDataset(object):
    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids,
            batch_first=True,
            padding_value=self.tokenizer.pad_token_id)
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=IGNORE_INDEX)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )


In [ ]:
# Create training and validation datasets using the custom dataset class
train_dataset = InstructDataset(train_dataset, tokenizer)
val_dataset = InstructDataset(val_dataset, tokenizer)
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)


In [ ]:
# Debug: Check data format
sample_data = train_dataset[1]

print("=" * 80)
print("Debuging: ")
print(sample_data)
print("-" * 80)
print(f"input_ids:\n{tokenizer.decode(sample_data['input_ids'])}")
# Filter out IGNORE_INDEX before decoding labels
z = [token for token in sample_data['labels'] if token != IGNORE_INDEX]
print("-" * 80)
print(f"labels:\n{tokenizer.decode(z)}")
print("=" * 80)


## Model Preparation

Then we have to apply some preprocessing to the model to prepare it for training. For that use the `prepare_model_for_kbit_training` method from PEFT.

In [ ]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from peft import LoraConfig, get_peft_model
# You can try differnt parameter-effient strategy for model trianing, for more info, please check https://github.com/huggingface/peft
config = LoraConfig(
    r=8,
    lora_alpha=8,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)

In [ ]:
from fastchat.conversation import get_conv_template
device = "cuda"
model.eval()

@torch.no_grad()
def generate(prompt):
    input_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors='pt').to(device)
    outputs = model.generate(input_ids, do_sample=False, max_new_tokens=2048)
    return tokenizer.decode(*outputs, skip_special_tokens=True)

prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=False  # 禁用 Qwen3 的 thinking 模式
)
response = generate(text)
print("-"*80)
print(response)

## Training

### General Training Hyperparameters

In [ ]:
# Set training parameters
training_arguments = transformers.TrainingArguments(
    output_dir="./checkpoints",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=2,
    optim='paged_adamw_32bit',
    save_steps=0,
    logging_steps=1,
    eval_steps=50,  # 每50步评估一次验证集
    eval_strategy="steps",  # 启用验证集评估（新版本使用 eval_strategy）
    learning_rate=2e-7,
    weight_decay=0.001,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine",
    gradient_checkpointing=True,
    report_to="none",
    load_best_model_at_end=False,  # 不加载最佳模型（节省内存）
    save_total_limit=1  # 只保留最新的checkpoint
)

In [ ]:
model.train()
trainer = transformers.Trainer(
    model=model,
    args=training_arguments,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator
)
# 为PEFT模型设置label_names，避免警告
trainer.label_names = ["labels"]


In [ ]:
model.print_trainable_parameters()

In [ ]:
trainer.train()

In [ ]:
# Plot training and validation loss curves with dual y-axes
import matplotlib.pyplot as plt
import pandas as pd
import os

print("=" * 80)
print("📊 绘制训练和验证损失曲线（双Y轴）...")

# Extract loss history from trainer
log_history = trainer.state.log_history

# Separate training and validation logs
train_logs = [log for log in log_history if 'loss' in log and 'eval_loss' not in log]
eval_logs = [log for log in log_history if 'eval_loss' in log]

# Extract data
train_steps = [log['step'] for log in train_logs]
train_loss = [log['loss'] for log in train_logs]

eval_steps = [log['step'] for log in eval_logs]
eval_loss = [log['eval_loss'] for log in eval_logs]

print(f"✅ 已提取 {len(train_logs)} 个训练loss记录")
print(f"✅ 已提取 {len(eval_logs)} 个验证loss记录")

# Create the plot with dual y-axes
fig, ax1 = plt.subplots(figsize=(12, 7))

# Left y-axis: Training Loss
color_train = 'tab:blue'
ax1.set_xlabel('Steps', fontsize=13, fontweight='bold')
ax1.set_ylabel('Training Loss', fontsize=13, fontweight='bold', color=color_train)
line1 = ax1.plot(train_steps, train_loss, 'o-', color=color_train, label='Training Loss', 
                 linewidth=2, markersize=4, alpha=0.8)
ax1.tick_params(axis='y', labelcolor=color_train)
ax1.grid(True, alpha=0.3, linestyle='--')

# Right y-axis: Validation Loss
if eval_loss:  # Only plot if validation loss exists
    ax2 = ax1.twinx()  # Create a second y-axis sharing the same x-axis
    color_val = 'tab:red'
    ax2.set_ylabel('Validation Loss', fontsize=13, fontweight='bold', color=color_val)
    line2 = ax2.plot(eval_steps, eval_loss, 's-', color=color_val, label='Validation Loss', 
                     linewidth=2, markersize=6, alpha=0.9)
    ax2.tick_params(axis='y', labelcolor=color_val)
    
    # Combine legends from both axes
    lines = line1 + line2
    labels = [l.get_label() for l in lines]
    ax1.legend(lines, labels, fontsize=12, loc='best', framealpha=0.9)
else:
    ax1.legend(fontsize=12, loc='best', framealpha=0.9)

plt.title('Training and Validation Loss Curves (Dual Y-Axes)', fontsize=15, fontweight='bold', pad=20)
plt.tight_layout()

# Save the plot
output_plot_path = '/kaggle/working/lora_models/qwen3_1_7b_custom/loss_curve.png'
os.makedirs(os.path.dirname(output_plot_path), exist_ok=True)
plt.savefig(output_plot_path, dpi=500, bbox_inches='tight')
print(f"✅ Loss curve saved to: {output_plot_path}")

plt.show()

# Print summary statistics
print("\n" + "=" * 80)
print("📈 Loss Statistics:")
print(f"   初始训练Loss: {train_loss[0]:.4f}")
print(f"   最终训练Loss: {train_loss[-1]:.4f}")
print(f"   Loss下降: {train_loss[0] - train_loss[-1]:.4f} ({(1 - train_loss[-1]/train_loss[0])*100:.2f}%)")
if eval_loss:
    print(f"   初始验证Loss: {eval_loss[0]:.4f}")
    print(f"   最终验证Loss: {eval_loss[-1]:.4f}")
    print(f"   验证Loss下降: {eval_loss[0] - eval_loss[-1]:.4f} ({(1 - eval_loss[-1]/eval_loss[0])*100:.2f}%)")
print("=" * 80)

Once the training is completed, we can evaluate our model and get its perplexity on the validation set like this:

In [ ]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

## Save Trained LoRA

In [ ]:
# Save trained LoRA model and tokenizer
output_path = "/kaggle/working/lora_models/qwen3_1_7b_custom"
os.makedirs(output_path, exist_ok=True)
trainer.save_model(output_path)
# 💾 同时保存tokenizer，确保加载时词汇表大小一致
tokenizer.save_pretrained(output_path)
print(f"✅ 模型已保存到: {output_path}")
print(f"✅ Tokenizer已保存到: {output_path}")
print(f"   词汇表大小: {len(tokenizer)}")

### Test the trained model

In [ ]:
from fastchat.conversation import get_conv_template
device = "cuda"
model.eval()
@torch.no_grad()
def generate(prompt):
    input_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors='pt').to(device)
    outputs = trainer.model.generate(input_ids, do_sample=False, max_new_tokens=1024)
    return tokenizer.decode(*outputs
    , skip_special_tokens=True)

prompt = "向前走1米"
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=False  # 禁用 Qwen3 的 thinking 模式
)
response = generate(text)
print("-"*80)
print(response)

# Clean GPU Memory and save lora zip

In [ ]:
# Empty VRAM
del model
del trainer
import gc
gc.collect()
gc.collect()

import os
import zipfile
import datetime

def file2zip(packagePath, zipPath):
    '''
  :param packagePath: 文件夹路径
  :param zipPath: 压缩包路径
  :return:
  '''
    zip = zipfile.ZipFile(zipPath, 'w', zipfile.ZIP_DEFLATED)
    for path, dirNames, fileNames in os.walk(packagePath):
        fpath = path.replace(packagePath, '')
        for name in fileNames:
            fullName = os.path.join(path, name)
            name = fpath + '\\' + name
            zip.write(fullName, name)
    zip.close()


if __name__ == "__main__":
    # 文件夹路径
    packagePath = '/kaggle/working/lora_models/qwen3_1_7b_custom'
    zipPath = '/kaggle/working/output.zip'
    if os.path.exists(zipPath):
        os.remove(zipPath)
    file2zip(packagePath, zipPath)
    print("打包完成")
    print(datetime.datetime.utcnow())


## Load the trained model back and integrate the trained LoRA within.

In [ ]:
# from peft import PeftModel
# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
# import os

# # 显存监控函数
# def get_memory_usage():
#     """获取当前GPU显存使用情况（MB）"""
#     if torch.cuda.is_available():
#         allocated = torch.cuda.memory_allocated() / 1024**2  # MB
#         reserved = torch.cuda.memory_reserved() / 1024**2   # MB
#         return allocated, reserved
#     return 0, 0

# def format_memory(mb):
#     """格式化显存显示"""
#     if mb < 1024:
#         return f"{mb:.2f} MB"
#     else:
#         return f"{mb/1024:.2f} GB"

# # 初始化显存监控
# torch.cuda.reset_peak_memory_stats() if torch.cuda.is_available() else None
# memory_stats = {
#     "initial": get_memory_usage(),
#     "after_tokenizer": None,
#     "after_base_model": None,
#     "after_resize": None,
#     "after_lora": None,
#     "after_merge": None,
#     "final": None
# }

# print("=" * 80)
# print("📊 GPU 显存监控")
# print("=" * 80)
# allocated, reserved = memory_stats["initial"]
# print(f"初始显存: 已分配 {format_memory(allocated)}, 已保留 {format_memory(reserved)}")

# # Re-define quantization parameters if needed, or use the ones defined earlier
# bnb_4bit_quant_type = "nf4" # or "fp4"
# use_nested_quant = True # or False

# model_id = "Qwen/Qwen3-1.7B"

# # Get the cache directory from environment variable
# transformers_cache_dir = os.environ.get('TRANSFORMERS_CACHE', '')

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=use_nested_quant,
#     bnb_4bit_quant_type=bnb_4bit_quant_type,
#     bnb_4bit_compute_dtype=torch.bfloat16
# )

# # Load trained LoRA model
# output_path = "/kaggle/working/lora_models/qwen3_1_7b_custom"
# print(f"\n📂 加载 LoRA 模型: {output_path}")

# # 🔧 步骤1: 先加载训练时保存的tokenizer（确保词汇表大小一致）
# print("\n🔄 加载训练时的 tokenizer...")
# tokenizer = AutoTokenizer.from_pretrained(output_path, padding_side="left")
# print(f"   ✅ Tokenizer 加载完成，词汇表大小: {len(tokenizer)}")

# # Fix: Set pad_token to avoid attention_mask warnings
# if tokenizer.pad_token is None:
#     tokenizer.pad_token = tokenizer.eos_token
#     tokenizer.pad_token_id = tokenizer.eos_token_id
#     print(f"   设置 pad_token = eos_token ({tokenizer.pad_token})")

# memory_stats["after_tokenizer"] = get_memory_usage()
# allocated, reserved = memory_stats["after_tokenizer"]
# print(f"   💾 显存: 已分配 {format_memory(allocated)}, 已保留 {format_memory(reserved)}")

# # 🔧 步骤2: 加载基础模型
# print("\n🔄 加载基础模型...")
# model = AutoModelForCausalLM.from_pretrained(
#     model_id, 
#     quantization_config=bnb_config,
#     device_map={"":0},
#     cache_dir=transformers_cache_dir
# )

# memory_stats["after_base_model"] = get_memory_usage()
# allocated, reserved = memory_stats["after_base_model"]
# print(f"   💾 显存: 已分配 {format_memory(allocated)}, 已保留 {format_memory(reserved)}")

# # 🔧 步骤3: Resize模型词汇表以匹配训练时的tokenizer
# print(f"\n🔄 Resize模型词汇表到: {len(tokenizer)}")
# model.resize_token_embeddings(len(tokenizer))

# memory_stats["after_resize"] = get_memory_usage()
# allocated, reserved = memory_stats["after_resize"]
# print(f"   💾 显存: 已分配 {format_memory(allocated)}, 已保留 {format_memory(reserved)}")

# # 🔧 步骤4: 加载LoRA适配器
# print("\n🔄 加载 LoRA 适配器...")
# model = PeftModel.from_pretrained(model, output_path)

# memory_stats["after_lora"] = get_memory_usage()
# allocated, reserved = memory_stats["after_lora"]
# print(f"   💾 显存: 已分配 {format_memory(allocated)}, 已保留 {format_memory(reserved)}")

# print("\n🔄 合并 LoRA 权重...")
# model = model.merge_and_unload()
# model.config.max_length = 512
# model.eval()

# memory_stats["after_merge"] = get_memory_usage()
# allocated, reserved = memory_stats["after_merge"]
# print(f"   💾 显存: 已分配 {format_memory(allocated)}, 已保留 {format_memory(reserved)}")

# memory_stats["final"] = get_memory_usage()

# # 获取峰值显存
# if torch.cuda.is_available():
#     peak_allocated = torch.cuda.max_memory_allocated() / 1024**2  # MB
#     peak_reserved = torch.cuda.max_memory_reserved() / 1024**2     # MB
# else:
#     peak_allocated = 0
#     peak_reserved = 0

# print("\n" + "=" * 80)
# print("📊 显存使用统计")
# print("=" * 80)
# print(f"初始显存:     已分配 {format_memory(memory_stats['initial'][0])}, 已保留 {format_memory(memory_stats['initial'][1])}")
# if memory_stats["after_tokenizer"]:
#     print(f"加载Tokenizer后: 已分配 {format_memory(memory_stats['after_tokenizer'][0])}, 已保留 {format_memory(memory_stats['after_tokenizer'][1])}")
# if memory_stats["after_base_model"]:
#     print(f"加载基础模型后: 已分配 {format_memory(memory_stats['after_base_model'][0])}, 已保留 {format_memory(memory_stats['after_base_model'][1])}")
# if memory_stats["after_resize"]:
#     print(f"Resize词汇表后: 已分配 {format_memory(memory_stats['after_resize'][0])}, 已保留 {format_memory(memory_stats['after_resize'][1])}")
# if memory_stats["after_lora"]:
#     print(f"加载LoRA适配器后: 已分配 {format_memory(memory_stats['after_lora'][0])}, 已保留 {format_memory(memory_stats['after_lora'][1])}")
# if memory_stats["after_merge"]:
#     print(f"合并LoRA权重后: 已分配 {format_memory(memory_stats['after_merge'][0])}, 已保留 {format_memory(memory_stats['after_merge'][1])}")
# print(f"最终显存:     已分配 {format_memory(memory_stats['final'][0])}, 已保留 {format_memory(memory_stats['final'][1])}")
# print("-" * 80)
# print(f"🔥 峰值显存使用: 已分配 {format_memory(peak_allocated)}, 已保留 {format_memory(peak_reserved)}")
# print(f"📈 显存增长: 已分配 {format_memory(memory_stats['final'][0] - memory_stats['initial'][0])}, 已保留 {format_memory(memory_stats['final'][1] - memory_stats['initial'][1])}")
# print("=" * 80)

# print("\n✅ 模型加载完成（词汇表大小已匹配）")

## Answer generation

In [ ]:
# from tqdm import tqdm
# @torch.no_grad()
# def generate(query_list, return_answer: bool = False):
#     def conv_format(query):
#         conv = get_conv_template('phoenix')
#         conv.append_message(conv.roles[0], query)
#         conv.append_message(conv.roles[1], None)
#         return conv.get_prompt()
#     query_list = [conv_format(query) for query in query_list]
#     input_ids = tokenizer(query_list, padding=True, truncation=True, return_tensors="pt", add_special_tokens=False).input_ids.to("cuda")
#     n_input, n_seq = input_ids.shape[0], input_ids.shape[-1]
#     output_ids = []
#     step = 1
#     for index in tqdm(range(0, n_input, step)):
#         outputs = model.generate(
#             input_ids=input_ids[index: min(n_input, index+step)],
#             do_sample=False,
#             max_new_tokens=512,
#             # temperature=0.3,
#             repetition_penalty=1.0,
#         )
#         output_ids += outputs
#     responses = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
#     if return_answer:
#         return [response[len(query):].strip() for query, response in zip(query_list, responses)]
#     return responses

# # test
# print("\n".join(generate(["What's the weather like today?", "Who are you?"])))

## Model Inference

Test the trained model with sample queries.

In [ ]:
# # Test the model with sample queries
# test_queries = [
#     "向右转15度，然后向前走0.5米。",
#     "给我站起来，然后向左转45度。",
#     "向前移动8米，接着右转120度，再走3米。"
# ]

# print("=" * 80)
# print("🧪 测试模型推理...")
# print("=" * 80)

# test_responses = generate(test_queries, return_answer=True)

# for i, (query, response) in enumerate(zip(test_queries, test_responses)):
#     print(f"\n查询 {i+1}:")
#     print(f"  问题: {query}")
#     print(f"  回答: {response[:200]}...")
#     print("-" * 80)

In [ ]:
# # Save test results
# output_file = "/kaggle/working/lora_models/qwen3_1_7b_custom/test_results.json"
# test_results = [
#     {"query": q, "response": r} 
#     for q, r in zip(test_queries, test_responses)
# ]
# with open(output_file, 'w', encoding='utf-8') as writer:
#     json.dump(test_results, writer, indent=4, ensure_ascii=False)
# print(f"\n✅ 测试结果已保存到: {output_file}")
# print(f"共 {len(test_results)} 条测试结果")